In [12]:
# Imprting the required packages to process raw data

import os

import numpy as np
import pandas as pd

#From processing_functions import check_leap_year
from processing_functions import get_digit_date
from lonlat2twd97 import lonlat2twd97


filenames = os.listdir('time_series_data')
datafoldername = 'time_series_data/'

export_folder = 'processed_timeseries'
try:
    os.mkdir(export_folder)
except:
    print('The folder "processed_timeseries" already exist. Continuing...')

sitenames = [i[0:-4] for i in filenames]


for curr_file in filenames:
    curr_file = filenames[5]
    sitename = curr_file[0:-4]
    data = pd.read_csv(datafoldername+curr_file, delim_whitespace=True, header=None)
    data.columns = ["date", "lon", "lat", "vert"]
    
# converting longitude latitude to local East (E) and North (N)
    
    E_97, N_97 = lonlat2twd97(data.lon, data.lat)
    E = (E_97 - np.min(E_97))*1000
    N = (N_97 - np.min(N_97))*1000
    U = (data.vert - np.min(data.vert))*1000     # convert from m to mm
    digit_date = get_digit_date(data.date)       # Processing date to digit unit
    data = data.assign(digit_date = digit_date)  # final processed data
    data = data.assign(E_mm=E,N_mm=N,U_mm=U)
    pd.DataFrame.to_csv(data,export_folder+'/'+sitename+'.csv')

The folder "processed_timeseries" already exist. Continuing...


In [13]:
## Computing GNSS (GPS) Time Series 

import os

from plotly import tools
import plotly.offline as pyo
import plotly.graph_objs as go

import numpy as np
import pandas as pd


filenames = os.listdir('processed_timeseries')
datafoldername = 'processed_timeseries/'

export_folder = 'timeseries_graphs'
try:
    os.mkdir(export_folder)
except:
    print('The folder "timeseries_graphs" already exist. Continuing...')

sitenames = [i[0:-4] for i in filenames]


for curr_file in filenames:
    curr_file = filenames[0]
    sitename = curr_file[0:-4]

data = pd.read_csv(datafoldername+curr_file)

traceE = go.Scatter(
    x = data.digit_date,
    y = data.E_mm,
    mode = 'lines+markers',
    name = sitename+'_E'
)

traceN = go.Scatter(
    x = data.digit_date,
    y = data.N_mm,
    mode = 'lines+markers',
    name = sitename+'_N',
    xaxis = 'x2',
    yaxis = 'y2'
)

traceU = go.Scatter(
    x = data.digit_date,
    y = data.U_mm,
    mode = 'lines+markers',
    name = sitename+'_U',
    xaxis = 'x3',
    yaxis = 'y3'
)

#data = go.Data([traceE,traceN,traceU])
data = [traceE,traceN,traceU]

layout = go.Layout(
title= sitename,
hovermode= 'closest',
showlegend= False,
height=900,
width=1100,
xaxis= dict(
#title= 'Year',
zeroline= False,
gridwidth= 1,
domain = [0,1]
),
yaxis= dict(
title= 'East (mm)',
gridwidth= 1,
domain = [0.7,1]
),
xaxis2= dict(
#title= 'Year222',
zeroline= False,
gridwidth= 1,
domain = [0,1],
anchor = 'y2'
),
yaxis2= dict(
title= 'North (mm)',
gridwidth= 1,
domain = [0.35,0.65],
anchor = 'x2'
),
xaxis3= dict(
title= 'Year',
zeroline= False,
gridwidth= 1,
domain = [0,1],
anchor = 'y3'
),
yaxis3= dict(
title= 'Vertical (mm)',
gridwidth= 1,
domain = [0,0.3],
anchor='x3'
),
)

fig = go.Figure(data = data, layout = layout)
pyo.plot(fig, filename=export_folder+'/'+sitename+'.html',auto_open=False)

The folder "timeseries_graphs" already exist. Continuing...


'timeseries_graphs/OLO8.html'

In [14]:
## Fitting GNSS (GPS) Time Series to Compute Velocity

import os

from plotly import tools
import plotly.offline as pyo
import plotly.graph_objs as go

import numpy as np
import pandas as pd

from fit_one_site import fit_one_site

filenames = os.listdir('processed_timeseries')
datafoldername = 'processed_timeseries/'

sitenames = [i[0:-4] for i in filenames]


# First making all time series w.r.t. to a specific station 

print('Here is all the stations in the folder:')
print(sitenames)
ref_site = input('Desired reference station ("None" if you want no ref):')

if ref_site!='None':
    ref_site_filename = datafoldername+ref_site+'.csv'
    ref_vel = fit_one_site(ref_site_filename)[4]
else:
    ref_vel = [0,0,0]  # if given 'None', make ref_vel all 0s so no ref given.

#ref_time,ref_Efit,ref_Nfit,ref_Ufit,ref_vel = fit_one_site(ref_site_filename)


# Specifying the name of the export folder 
export_folder = 'fitted_timeseries_graphs_ref_to_'+ref_site
try:
    os.mkdir(export_folder)
except:
    print('The folder "'+export_folder+'" already exist. Continuing...')


## create empty variables to store velocity for future use
sitenames=[]
lon = []
lat = []
vel_E = []
vel_N = []
vel_U = []

for curr_file in filenames:
    curr_file = filenames[-4]
    sitename = curr_file[0:-4]
    print(sitename)
   
    data = pd.read_csv(datafoldername+curr_file)


# Reference to given station 

    time = data.digit_date
    E = data.E_mm-(time*ref_vel[0])
    N = data.N_mm-(time*ref_vel[1])
    U = data.U_mm-(time*ref_vel[2])


## Performing Linear fitting to compute velocity

# East (E) component 

# s = G*m
# s is the displacement component
# m is the parameters (slope, intercept)
# G is the Green's function (time, ones)

    G = np.vstack([time,np.ones(len(time))]).T
    sE = np.vstack(E)

    [slopeE,interceptE],sum_residualE = np.linalg.lstsq(G,sE,rcond=None)[0:2]

#print(slopeE,interceptE)

    traceE_fit = go.Scatter(
        x = time,
        y = slopeE*time+interceptE,
        mode = 'lines+text',
        line = dict(
                width = 2,
                color = 'rgba(0,0,255,.9)'
                ),
    )

    trace_print_Evel = go.Scatter(
        x = [time[round(len(time)/2)]],
        y = [np.max(sE)],
        mode = 'text',
        text = ['velocity = '+str(slopeE)[1:-1]+' mm/yr'],
        textposition = 'bottom center'
        )

# North (N) component 

    G = np.vstack([time,np.ones(len(time))]).T
    sN = np.vstack(N)

    [slopeN,interceptN],sum_residualN = np.linalg.lstsq(G,sN,rcond=None)[0:2]

#print(slopeN,interceptN)

    traceN_fit = go.Scatter(
        x = time,
        y = slopeN*time+interceptN,
        mode = 'lines+text',
        line = dict(
                width = 2,
                color = 'rgba(0,0,255,.9)'
                ),
        xaxis = 'x2',
        yaxis = 'y2'
        )

    trace_print_Nvel = go.Scatter(
        x = [time[round(len(time)/2)]],
        y = [np.max(sN)],
        mode = 'text',
        text = ['velocity = '+str(slopeN)[1:-1]+' mm/yr'],
        textposition = 'bottom center',
        xaxis = 'x2',
        yaxis = 'y2'
        )

# Up or Vertical (U) component 

    G = np.vstack([time,np.ones(len(time))]).T
    sU = np.vstack(U)

    [slopeU,interceptU],sum_residualU = np.linalg.lstsq(G,sU,rcond=None)[0:2]

#print(slopeU,interceptU)

    traceU_fit = go.Scatter(
        x = time,
        y = slopeU*time+interceptU,
        mode = 'lines+text',
        line = dict(
                width = 2,
                color = 'rgba(0,0,255,.9)'
                ),
        xaxis = 'x3',
        yaxis = 'y3'
        )

    trace_print_Uvel = go.Scatter(
        x = [time[round(len(time)/2)]],
        y = [np.max(sU)],
        mode = 'text',
        text = ['velocity = '+str(slopeU)[1:-1]+' mm/yr'],
        textposition = 'bottom center',
        xaxis = 'x3',
        yaxis = 'y3'
        )

## Saving the results

    sitenames.append(sitename)
    lon.append(data.lon[0]) # just get the first obs of lon lat
    lat.append(data.lat[0]) # just get the first obs of lon lat
    vel_E.append(slopeE[0]) # because slope is stored in two layers of []
    vel_N.append(slopeN[0])
    vel_U.append(slopeU[0])


## Plotting Time series 

    traceE = go.Scatter(
        x = time,
        y = E,
        mode = 'lines+markers',
        name = sitename+'_E',
        marker = dict(
                size = 6,
                color = 'rgba(255, 0, 0, .9)',
                line = dict(
                    width = 0,
                    color = 'rgb(0, 0, 0)'
                    )
                ),
        line = dict(
                width = 0.75,
                color = 'rgba(255,0,0,.9)'
                )
    )

    traceN = go.Scatter(
        x = time,
        y = N,
        mode = 'lines+markers',
        name = sitename+'_N',
        marker = dict(
                size = 6,
                color = 'rgba(255, 0, 0, .9)',
                line = dict(
                    width = 0,
                    color = 'rgb(0, 0, 0)'
                    )
                ),
        line = dict(
                width = 0.75,
                color = 'rgba(255,0,0,.9)'
                ),
        xaxis = 'x2',
        yaxis = 'y2'
    )

    traceU = go.Scatter(
        x = time,
        y = U,
        mode = 'lines+markers',
        name = sitename+'_U',
        marker = dict(
                size = 6,
                color = 'rgba(255, 0, 0, .9)',
                line = dict(
                    width = 0,
                    color = 'rgb(0, 0, 0)'
                    )
                ),
        line = dict(
                width = 0.75,
                color = 'rgba(255,0,0,.9)'
                ),
        xaxis = 'x3',
        yaxis = 'y3'
    )

#data = go.Data([traceE,traceN,traceU])
    data_fig = [traceE,traceN,traceU,traceE_fit,traceN_fit,traceU_fit]
    data_fig.append(trace_print_Evel)
    data_fig.append(trace_print_Nvel)
    data_fig.append(trace_print_Uvel)

    layout = go.Layout(
                title= sitename,
                hovermode= 'closest',
                showlegend= False,
                height=900,
                width=1100,
                xaxis= dict(
                            #title= 'Year',
                            zeroline= False,
                            gridwidth= 1,
                            domain = [0,1]
                            ),
                yaxis= dict(
                            title= 'East (mm)',
                            gridwidth= 1,
                            domain = [0.7,1]
                            ),
                xaxis2= dict(
                            #title= 'Year222',
                            zeroline= False,
                            gridwidth= 1,
                            domain = [0,1],
                            anchor = 'y2'
                            ),
                yaxis2= dict(
                            title= 'North (mm)',
                            gridwidth= 1,
                            domain = [0.35,0.65],
                            anchor = 'x2'
                            ),
                xaxis3= dict(
                            title= 'Year',
                            zeroline= False,
                            gridwidth= 1,
                            domain = [0,1],
                            anchor = 'y3'
                            ),
                yaxis3= dict(
                            title= 'Vertical (mm)',
                            gridwidth= 1,
                            domain = [0,0.3],
                            anchor='x3'
                            )
                )

    fig = go.Figure(data = data_fig, layout = layout)
    pyo.plot(fig, filename=export_folder+'/'+sitename+'.html',auto_open=False)


df = pd.DataFrame( {'site': pd.Series(sitenames, dtype=str), 
                    'lon': pd.Series(lon, dtype=float),
                    'lat': pd.Series(lat, dtype=float),
                    'vel_E': pd.Series(vel_E, dtype=float),
                    'vel_N': pd.Series(vel_N, dtype=float),
                    'vel_U': pd.Series(vel_U, dtype=float)})

pd.DataFrame.to_csv(df,'fitted_velocity_ref_to_'+ref_site+'.csv')


print(df.head(10))
print(df.site[0])
print(df.lon[0])
print(df.lat[0])
print(df.vel_E[0])
print(df.vel_E[0]+1)


Here is all the stations in the folder:
['OLO8', 'OLO9', 'Untitled.i', 'dat', 'OLO7', '.ipynb_checkpo', 'OLO6', 'OLO5', 'OLO1', 'OLO3']


Desired reference station ("None" if you want no ref): None


The folder "fitted_timeseries_graphs_ref_to_None" already exist. Continuing...
OLO6
OLO6
OLO6
OLO6
OLO6
OLO6
OLO6
OLO6
OLO6
OLO6
   site        lon      lat      vel_E      vel_N     vel_U
0  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
1  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
2  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
3  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
4  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
5  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
6  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
7  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
8  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
9  OLO6  35.905514 -2.70871  25.854741  18.778422 -2.848801
OLO6
35.9055140418
-2.7087103927
25.854741321636805
26.854741321636805
